In [ ]:
# default_exp core.transfer_model

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Transfer learning temporal model 
> Trains the temporal model from the Video Pose3D checkpoint made for inference. 

In [ ]:
#export
import time
import numpy as np
import torch

from runningpose.core.runningpose_dataset import RunningposeDataset
from runningpose.core.model import TemporalModel

#### Load Data
We need our 2D data to match the 3D ground truth data. 

In [ ]:
#export
# print('Loading dataset...')
# dataset_path = ''
# dataset = RunningposeDataset(dataset_path)

# Convert to 3D camera coordinates 
# TODO: Maybe just do this in format qtmdata. 

In [ ]:
print('Loading 2D detections...')
keypoints = np.load('data_2d_custom_tindra.npz', allow_pickle=True)
keypoints_metadata = keypoints['metadata'].item()
keypoints_symmetry = keypoints_metadata['keypoints_symmetry']
kps_left, kps_right = list(keypoints_symmetry[0]), list(keypoints_symmetry[1])
# joints_left, joints_right = list(dataset.skeleton().joints_left()), list(dataset.skeleton().joints_right())
keypoints = keypoints['positions_2d'].item()
print(keypoints.keys())
print(keypoints['Tindra_dynamic0001_Miqus_16_23604.avi'].keys())
print(keypoints['Tindra_dynamic0001_Miqus_16_23604.avi']['custom'][0].shape)

Loading 2D detections...
dict_keys(['Tindra_dynamic0001_Miqus_16_23604.avi', 'Tindra_dynamic0001_Miqus_15_21386.avi'])
dict_keys(['custom'])
(694, 17, 2)


We can see that we have a dictionary with all the videos and then a 
custom dictionary with a list with an array with (frames, keypoints, dim) 

#### Define the model
> Load the temporal model trained model for generating 3D predictions.

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# Load checkpoint
print('Loading checkpoint')
checkpoint = torch.load('pretrained_h36m_detectron_coco.bin', 
                        map_location=lambda storage,
                        loc: storage)
print('This model was trained for {} epochs'.format(checkpoint['epoch']))
checkpoint.keys()

Loading checkpoint
This model was trained for 80 epochs


dict_keys(['epoch', 'lr', 'model_pos'])

In [ ]:
# Load the model HARD CODED
num_joints_in = 17 # COCO
in_features = 2 # dimension of in joints
num_joints_out = 20 # runningpose
filter_widths = [3, 3, 3, 3, 3] # just as in inference  
causal = False # Not real time predictions 
dropout = 0.25 # default
channels = 1024 # default

model_pos = TemporalModel(
    num_joints_in, in_features, num_joints_out, filter_widths, causal, 
    dropout, channels
)
if torch.cuda.is_available():
    model_pos = model_pos.cuda()

In [ ]:
model_pos.load_state_dict(checkpoint['model_pos'])

<All keys matched successfully>

#### Training the model

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()